# Using Kqlmagic to analyze Azure SQL logs and metrics

There are two main tables in Azure Log Analytics (Azure Monitor Logs) workspace that capture Azure SQL events:
1. AzureDiagnostics
2. AzureMetric


## 1\. Connect to Azure Monitor Logs (Log Analytics) workspace

Workspace is similar to what a database is to SQL. You connect to Log Analytics workspace to start querying data.

### 1.1 Install Azure Monitor Logs extension first

Go to the Extension viewlet and type in Azure Monitor Logs. Install it and restart ADS.

### 1.2 Connect to the desired Azure Monitor Logs workspace

Change the Kernel to "Log Analytics". Set Attach to to a new or existing connection to the workspace. Note: you will need a workspace Id that you can obtain from Azure portal.

## 2. Analyze events by Diagnostic Settings

Let's do a simple query first to analyze the number of events by Operation Name. 

> **Note**: Each row in AzureDiagnostic represents an event for specific Operation or category. Some SQL actions may result in generating multiple events of different types.



In [1]:
AzureDiagnostics
| summarize count() by OperationName

Commands completed successfully.

Total execution time: 00:00:00.581

OperationName,count_
AuditEvent,1105
ErrorEvent,10
QueryStoreRuntimeStatisticsEvent,32
DatabaseWaitStatistcsEvent,9



The above query's equivalent in SQL is:
```
SELECT COUNT(*) AS [count_]
FROM AzureDiagnostics
GROUP BY OperationName
```


Count my Azure SQL DB events by category / diagnostic settings.

In [1]:
AzureDiagnostics
| where LogicalServerName_s == "jukoesmasqldb"
| where TimeGenerated >= ago(5d)
| summarize count() by Category
| render barchart with (title = "Azure SQL DB Diagnostic Category")

Commands completed successfully.

Total execution time: 00:00:00.336

Category,count_
SQLSecurityAuditEvents,290
Errors,8
QueryStoreRuntimeStatistics,12
DatabaseWaitStatistics,4


## 3. Performance troubleshooting Query (from Azure Portal)

Potentially a query or deadlock on the system that could lead to poor performance. The following is a query suggested by Azure Portal.

In [3]:
AzureMetrics
| where ResourceProvider == "MICROSOFT.SQL"
| where TimeGenerated >=ago(60min)
| where MetricName in ('deadlock')
| parse _ResourceId with * "/microsoft.sql/servers/" Resource // subtract Resource name for _ResourceId
| summarize Deadlock_max_60Mins = max(Maximum) by Resource, MetricName

Commands completed successfully.

Total execution time: 00:00:00.424

Resource,MetricName,Deadlock_max_60Mins


# AzureMetrics

This is a sample query to dig into AzureMetrics

In [2]:
AzureMetrics
| project-away TenantId, ResourceId, SubscriptionId, _ResourceId, ResourceGroup // hide sensitive info
| project TimeGenerated, MetricName, Total, Count, UnitName
| take 10

Commands completed successfully.

Total execution time: 00:00:00.675

TimeGenerated,MetricName,Total,Count,UnitName
2021-07-17 12:21:00,storage_percent,0,1,Percent
2021-07-17 12:21:00,physical_data_read_percent,0,4,Percent
2021-07-17 12:21:00,log_write_percent,0,4,Percent
2021-07-17 12:21:00,allocated_data_storage,33554432,1,Bytes
2021-07-17 12:21:00,sessions_percent,0,4,Percent
2021-07-17 12:21:00,cpu_used,0,1,Count
2021-07-17 12:21:00,cpu_limit,10,5,Count
2021-07-17 12:21:00,xtp_storage_percent,0,4,Percent
2021-07-17 12:21:00,cpu_percent,0,4,Percent
2021-07-17 12:21:00,storage,29097984,1,Bytes


# AzureDiagnostics

This is a sample query to dig into AzureDiagnostics. This table tends to have more details than AzureMetrics.

In [1]:
AzureDiagnostics
| project-away TenantId, ResourceId, SubscriptionId, ResourceGroup, _ResourceId // Hide sensitive columns :) 
| project TimeGenerated, Category, OperationName
| take 10

Commands completed successfully.

Total execution time: 00:00:00.307

TimeGenerated,Category,OperationName
2021-07-08 20:22:25,SQLSecurityAuditEvents,AuditEvent
2021-07-08 20:55:53,QueryStoreRuntimeStatistics,QueryStoreRuntimeStatisticsEvent
2021-07-08 22:04:55,SQLSecurityAuditEvents,AuditEvent
2021-07-08 22:04:55,SQLSecurityAuditEvents,AuditEvent
2021-07-08 21:10:49,SQLSecurityAuditEvents,AuditEvent
2021-07-08 21:10:13,QueryStoreRuntimeStatistics,QueryStoreRuntimeStatisticsEvent
2021-07-08 21:02:12,SQLSecurityAuditEvents,AuditEvent
2021-07-08 21:25:06,SQLSecurityAuditEvents,AuditEvent
2021-07-08 21:25:06,SQLSecurityAuditEvents,AuditEvent
2021-07-08 22:15:35,SQLSecurityAuditEvents,AuditEvent


## Analyze (non-audit) Events

In [15]:
AzureDiagnostics
| summarize event_count = count() by bin(TimeGenerated, 2d), OperationName
| where OperationName <> "AuditEvent"
| evaluate pivot(OperationName, sum(event_count))
| sort by TimeGenerated asc

Commands completed successfully.

Total execution time: 00:00:00.579

TimeGenerated,DatabaseWaitStatistcsEvent,ErrorEvent,QueryStoreRuntimeStatisticsEvent
2021-06-17 17:00:00,0,0,2
2021-06-21 17:00:00,4,0,9
2021-06-23 17:00:00,0,0,2
2021-06-29 17:00:00,1,2,2
2021-07-01 17:00:00,0,0,2
2021-07-05 17:00:00,0,0,1
2021-07-07 17:00:00,0,0,2
2021-07-13 17:00:00,4,0,9
2021-07-15 17:00:00,0,8,3


In [6]:
AzureDiagnostics
| summarize event_count=count() by bin(TimeGenerated, 2d), OperationName
| where OperationName <> "AuditEvent"

Commands completed successfully.

Total execution time: 00:00:00.417

TimeGenerated,OperationName,event_count
2021-07-15 17:00:00,ErrorEvent,8
2021-07-15 17:00:00,QueryStoreRuntimeStatisticsEvent,3
2021-07-07 17:00:00,QueryStoreRuntimeStatisticsEvent,2
2021-06-21 17:00:00,QueryStoreRuntimeStatisticsEvent,9
2021-06-21 17:00:00,DatabaseWaitStatistcsEvent,4
2021-06-17 17:00:00,QueryStoreRuntimeStatisticsEvent,2
2021-07-01 17:00:00,QueryStoreRuntimeStatisticsEvent,2
2021-06-23 17:00:00,QueryStoreRuntimeStatisticsEvent,2
2021-06-29 17:00:00,QueryStoreRuntimeStatisticsEvent,2
2021-06-29 17:00:00,DatabaseWaitStatistcsEvent,1


## Deadlock Analysis

In [18]:
AzureDiagnostics
| where OperationName == "DeadlockEvent"
| project TimeGenerated, Category, Resource, OperationName, Type, deadlock_xml_s
| sort by TimeGenerated desc
| take 50

Commands completed successfully.

Total execution time: 00:00:00.297

TimeGenerated,Category,Resource,OperationName,Type,deadlock_xml_s


Find the deadlock query plan

In [17]:
AzureDiagnostics
| where OperationName == "DeadlockEvent"
| extend d = parse_xml(deadlock_xml_s)
| project TimeGenerated, QueryPlanHash = d.deadlock.["process-list"].process[0].executionStack.frame[0]["@queryplanhash"], QueryHash = d.deadlock.["process-list"].process[0].executionStack.frame[0]["@queryhash"]
| take 50

Commands completed successfully.

Total execution time: 00:00:00.817

TimeGenerated,QuerhPlanHash,QueryHash


## Query Store Runtime Statistics Events

In [19]:
AzureDiagnostics
| where OperationName == "QueryStoreRuntimeStatisticsEvent"
| project TimeGenerated, query_hash_s, statement_sql_handle_s, query_plan_hash_s
| take 10

Commands completed successfully.

Total execution time: 00:00:00.681

TimeGenerated,query_hash_s,statement_sql_handle_s,query_plan_hash_s
2021-06-17 21:31:47,0xD0B172EC3AC90AB2,0x0900F0816A25DA2ADDAACB957F82B882EC160000000000000000000000000000000000000000000000000000,0x07C5C286730ED585
2021-06-17 21:27:36,0xD0B172EC3AC90AB2,0x0900F0816A25DA2ADDAACB957F82B882EC160000000000000000000000000000000000000000000000000000,0x2B9A3A3B55B4B6CB
2021-06-21 23:49:13,0xE9A90E6DF072038B,0x09003A8994953FDD36568246F07679879BA60000000000000000000000000000000000000000000000000000,0x034C49FD4D7CD726
2021-06-21 23:49:13,0x1952171B581C894D,0x0900506DB841E46B2920275FF14C13E8BFC70000000000000000000000000000000000000000000000000000,0xB496791006554667
2021-06-21 23:49:13,0x1952171B581C894D,0x0900FA2BFA7C760AF0B2E9D5ED64A043458D0000000000000000000000000000000000000000000000000000,0xF3695F01D8FE0E91
2021-06-21 23:49:13,0x5D10B550067140CF,0x0900C2E90943F6588EBCEBF060DC800B68AB0000000000000000000000000000000000000000000000000000,0x754D0416154A71D1
2021-06-21 23:49:13,0x82C4887635CEB06D,0x09006EFFD14D9A2EA5112AD9C2FCDB049B200000000000000000000000000000000000000000000000000000,0xA0268D84A1CF355E
2021-06-21 23:49:13,0xADCF2ECC6A4FE6E4,0x090048201C1EC9E32104315E4CD4D45627F90000000000000000000000000000000000000000000000000000,0x54B4226B7D07F982
2021-06-21 23:49:13,0x6FABBDFDEF26BD6E,0x09005D81DADE14DA3419C53961A61D5563170000000000000000000000000000000000000000000000000000,0xEA952D63A06CA802
2021-06-21 23:49:13,0x2BAD38CE65D68382,0x0900AAB3AB4F6E5FC9AA33F1E8C27322462F0000000000000000000000000000000000000000000000000000,0x1EE62D991B0D0DFC


## Analyze Errors

In [3]:
AzureDiagnostics
| where OperationName == "ErrorEvent"
| extend ErrorNumber =  tostring(error_number_d) 
| summarize event_count=count() by EventTime = bin(TimeGenerated, 2d),  ErrorNumber
| evaluate pivot(ErrorNumber, sum(event_count))
| sort by EventTime asc


Commands completed successfully.

Total execution time: 00:00:00.279

EventTime,208.0,3902.0
2021-06-29 17:00:00,2,0
2021-07-15 17:00:00,27,1
2021-07-17 17:00:00,20,1
2021-07-19 17:00:00,6,0


## Find Deleted table

In [16]:
AzureDiagnostics
| where action_name_s in ('BATCH COMPLETED')
| project TimeGenerated, Category, action_name_s, statement_s
| where statement_s contains "DROP TABLE"
| sort by TimeGenerated desc 
| take 10



Commands completed successfully.

Total execution time: 00:00:01.604

TimeGenerated,Category,action_name_s,statement_s
